In [1]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import os
from scipy import misc
from  collections import Counter


Using TensorFlow backend.


In [2]:
PATH_TO_IMAGES="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/image_10_resized/"
PATH_TO_LABELS="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/files/_all_in_one_10.csv"

### Работа с данными

In [28]:
#Загрузка данных
X=np.load(PATH_TO_IMAGES+"X_data.npy")
y=np.load(PATH_TO_IMAGES+"y_data.npy")

In [31]:
#Кодирую имена числами
la=LabelEncoder()
y_copy=y.copy()
y=la.fit_transform(y)

In [32]:
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=17,stratify=y)

In [33]:
num_train, height, width,depth = X_train.shape 
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0] 


Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

### Создание архитектуры

In [34]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = (3,3) # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [35]:
inp = Input(shape=(height, width,depth))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Conv2D(conv_depth_1, kernel_size,  padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(drop_1)
conv_4 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

### Обучение модели

In [36]:
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation


Train on 10076 samples, validate on 1120 samples
Epoch 1/200
  128/10076 [..............................] - ETA: 26:13 - loss: 16.1181 - acc: 0.0000e+00

KeyboardInterrupt: 

### Проверка качества модели

In [37]:
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 207s 43ms/step


[15.581926462078473, 0.00062513023546572201]